## Abstract

First Data for Passive Data Collection using Smartwatches and GPS from the PREACT Study. 

## Introduction

Treatment personalization is highly discussed to counteract insufficient response rates in psychotherapy. In the quest for criteria allowing informed selection or adaptation, ambulatory assessment data (i.e. EMA, passive sensing)are a key component, as processes happening outside of therapy sessions can be depicted in high temporal and/or spatial resolution.

PREACT is a multicenter prospective-longitudinal study investigating different predictors of non-response (i.e. EEG, fMRI) in around 500 patients undergoing cognitive behavioral therapy for internalizing disorders (https://forschungsgruppe5187.de/de). 

## Methods
Patients can enroll for therapy-accompanying ambulatory assessment. They are provided with a customized study app and a state-of-the-art smartwatch collecting passive data like GPS and heart rate for up to 365 days. In parallel, three 14-day EMA phases (pre-, mid- and post-therapy) cover transdiagnostic (i.e. emotion regulation), contextual and therapy-related aspects.  

Here, we present first results on data compliance and quality for the passive sensing data as well as EMA assessments.


In [1]:
import os
import glob
import pickle

import pandas as pd
import datetime as dt
from datetime import date, datetime
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


from config import datapath, proj_sheet

today = "22042024"

# passive + ema_data
datapath1 = datapath + f"export_tiki_{today}/"
file_pattern = os.path.join(datapath1, "epoch_part*.csv")
file_list = glob.glob(file_pattern)
file_list.sort()

df_complete = pd.concat((pd.read_csv(f, encoding="latin-1", low_memory=False) for f in file_list), ignore_index=True)
today = "15042024"
df_redcap_zert = pd.read_csv(datapath + f"ZERTIFIZIERUNGFOR518_DATA_{today}.csv")
df_redcap = pd.read_csv(datapath + f"FOR5187_DATA_{today}.csv")
df_monitoring = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{proj_sheet}/export?format=csv")


In [2]:
df_redcap_kosten = pd.read_csv(datapath + "Kosten_2024-04-15.csv")
df_redcap_zert_kosten = pd.read_csv(datapath + "ZertKosten_2024-04-15.csv")

df_redcap_kosten = df_redcap_kosten[['for_id','costs_ema_min', 'costs_ema_burden', 'costs_ema_text']]
df_redcap_zert_kosten = df_redcap_zert_kosten[['for_id','costs_ema_min', 'costs_ema_burden', 'costs_ema_text']]

df_redcap_kosten = pd.concat([df_redcap_kosten, df_redcap_zert_kosten],ignore_index=True)


In [3]:
df_redcap_kosten.dropna(subset= "costs_ema_burden", inplace=True)

In [4]:
bsi_scales = {
    'bsi_somatization': ['bsi_2', 'bsi_7', 'bsi_23', 'bsi_29', 'bsi_30', 'bsi_33', 'bsi_37'],
    'bsi_compulsivity': ['bsi_5', 'bsi_15', 'bsi_26', 'bsi_27', 'bsi_32', 'bsi_36'],
    'bsi_insecurity': ['bsi_20', 'bsi_21', 'bsi_22', 'bsi_42'],
    'bsi_depression': ['bsi_9', 'bsi_16', 'bsi_17', 'bsi_18', 'bsi_35', 'bsi_50'],
    'bsi_anxiety': ['bsi_1', 'bsi_12', 'bsi_19', 'bsi_38', 'bsi_45', 'bsi_49'],
    'bsi_aggression': ['bsi_6', 'bsi_13', 'bsi_40', 'bsi_41', 'bsi_46'],
    'bsi_phobia': ['bsi_8', 'bsi_28', 'bsi_31', 'bsi_43', 'bsi_47'],
    'bsi_paranoia': ['bsi_4', 'bsi_10', 'bsi_24', 'bsi_48', 'bsi_51'],
    'bsi_psychotizism': ['bsi_3', 'bsi_14', 'bsi_34', 'bsi_44', 'bsi_53'],
    'bsi_additional': ['bsi_11', 'bsi_25', 'bsi_39', 'bsi_52']
}

# Check for missing columns and calculate each scale
for scale, columns in bsi_scales.items():
    missing_cols = [col for col in columns if col not in df_redcap_zert.columns]
    if missing_cols:
        print(f"Missing columns in DataFrame for {scale}:", missing_cols)
    else:
        # Sum the specified columns and create a new column for each scale
        df_redcap_zert[scale] = df_redcap_zert[columns].sum(axis=1)

In [5]:
bsi_columns = [f'bsi_{i}' for i in range(1, 54)]
df_redcap_zert['bsi_gs'] = df_redcap_zert[bsi_columns].sum(axis=1)
df_redcap_zert['bsi_gsi'] = df_redcap_zert["bsi_gs"]/53


In [6]:
df_redcap_zert = df_redcap_zert[['for_id', 'ema_start_date','ema_smartphone', 'ema_special_event', 'age', 'gender', 'marital_status', 'partnership', 'graduation','profession','years_of_education',
 'employability', 'scid_cv_date','ses', 'somatic_problems','scid_cv_prim_cat', 'bsi_date','bsi_somatization',
 'bsi_compulsivity','bsi_insecurity','bsi_depression','bsi_anxiety','bsi_aggression','bsi_phobia','bsi_paranoia',
 'bsi_psychotizism','bsi_additional','bsi_gs','bsi_gsi', 'prior_treatment']] #ema_wear_exp

### Redcap data

In [7]:
df_redcap = pd.concat([df_redcap, df_redcap_zert],ignore_index=True)

df_redcap = df_redcap[['for_id', 'ema_start_date','ema_smartphone',
 'ema_wear_exp', 'ema_special_event', 'age', 'gender', 'marital_status', 'partnership', 'graduation','profession','years_of_education',
 'employability','ses', 'somatic_problems','scid_cv_prim_cat','bsi_somatization', 'prior_treatment',
 'bsi_compulsivity','bsi_insecurity','bsi_depression','bsi_anxiety','bsi_aggression','bsi_phobia','bsi_paranoia',
 'bsi_psychotizism','bsi_additional','bsi_gs','bsi_gsi']] 

In [8]:
df_redcap["ema_start_date"] = pd.to_datetime(df_redcap["ema_start_date"])

In [9]:
gender_mapping = {
    1: 'male',
    2: 'female',
    3: 'diverse',
    4: 'no gender',
    5: 'not specified'
}

scid_cv_cat_mapping = {
    1: 'Depressive Disorder',
    2: 'Specific Phobia',
    3: 'Social Anxiety Disorder',
    4: 'Agoraphobia and/or Panic Disorder',
    5: 'Generalized Anxiety Disorder',
    6: 'Obsessive-Compulsive Disorder',
    7: 'Post-Traumatic Stress Disorder'
}

marital_status_mapping = {
    1: 'single',
    2: 'married/registered partnership',
    3: 'divorced',
    4: 'separated',
    5: 'widowed',
    6: 'other'
}

employability_mapping = {
    0: 'employable',
    1: 'unemployable (on sick leave)',
    2: 'on disability pension',
    3: 'on retirement pension',
    4: 'other'
}

graduation_mapping = {
    0: 'still in school',
    1: 'no school degree',
    2: 'elementary school degree or equivalent',
    3: 'middle school degree or equivalent',
    4: 'high school diploma/university entrance qualification',
    5: 'other'
}

profession_mapping = {
    0: 'still in training or studies',
    1: 'no training degree',
    2: 'vocational training, including technical school',
    3: 'university or college degree',
    4: 'other'
}

prior_treatment_mapping = {
    0: 'no prior treatment',
    1: 'outpatient psychotherapy',
    2: 'inpatient or partial inpatient treatment/psychotherapy',
    3: 'both',
    4: 'yes'
}

ema_smartphone_mapping = {
    1: 'iPhone',
    0: 'Android'
}

ema_special_event_mapping = {
    0: 'usual',
    1: 'special event'
}


In [10]:
# Apply mappings
df_redcap['gender_description'] = df_redcap['gender'].map(gender_mapping)
df_redcap['scid_cv_description'] = df_redcap['scid_cv_prim_cat'].map(scid_cv_cat_mapping)
df_redcap['marital_status_description'] = df_redcap['marital_status'].map(marital_status_mapping)
df_redcap['employability_description'] = df_redcap['employability'].map(employability_mapping)
df_redcap['graduation_description'] = df_redcap['graduation'].map(graduation_mapping)
df_redcap['profession_description'] = df_redcap['profession'].map(profession_mapping)
df_redcap['prior_treatment_description'] = df_redcap['prior_treatment'].map(prior_treatment_mapping)
df_redcap['ema_smartphone_description'] = df_redcap['ema_smartphone'].map(ema_smartphone_mapping)
df_redcap['ema_special_event_description'] = df_redcap['ema_special_event'].map(ema_special_event_mapping)

### Monitoring data

In [11]:
df_monitoring = df_monitoring.copy()
df_monitoring.rename(columns = {"Pseudonym": "customer", "EMA_ID": "ema_id", "Status": "status",
                                "Studienversion":"study_version", "FOR_ID":"for_id", 
                           "Start EMA Baseline": "ema_base_start", "Ende EMA Baseline": "ema_base_end", 
                           "Freischaltung/ Start EMA T20": "ema_t20_start","Ende EMA T20":"ema_t20_end", 
                               "Termin 1. Gespräch": "first_call_date"}, inplace=True)

In [12]:
df_monitoring["customer"] = df_monitoring["customer"].str[:4]
df_monitoring = df_monitoring[["customer", "ema_id", "study_version", "for_id", "status"]]
df_monitoring["for_id"] = df_monitoring.for_id.str.strip()



In [13]:
df_monitoring = pd.merge(df_monitoring, df_redcap, on="for_id", how="left")
df_monitoring = pd.merge(df_monitoring, df_redcap_kosten, on="for_id", how="left")


### Passive data

In [14]:
df_complete.timezoneOffset.unique()

array([  60.,   nan,  420.,  120.,  180.,    0.,  480., -300., -360.,
       -420.,  330., -240.,  360.,  -60.,  540., -180.])

In [15]:
### 1.1 Import epoch level passive + GPS data

df_complete["customer"] = df_complete.customer.str.split("@").str.get(0)
df_complete["customer"] = df_complete["customer"].str[:4]

df_complete["startTimestamp"] = pd.to_datetime(df_complete["startTimestamp"],unit='ms')
df_complete["createdAt"] = pd.to_datetime(df_complete["createdAt"],unit='ms')

## Timezone offset miteinbeziehen?

# Handle NaN in timezoneOffset by skipping or replacing with a default
# Here, we skip adjustments for NaN offsets by using np.where to check for NaN
df_complete['startTimestamp'] = np.where(df_complete['timezoneOffset'].isna(),
                               df_complete['startTimestamp'],  # If NaN, keep original timestamp
                               df_complete['startTimestamp'] + pd.to_timedelta(df_complete['timezoneOffset'], unit='m'))  # Else, apply offset

df_complete['createdAt'] = np.where(df_complete['createdAt'].isna(),
                               df_complete['createdAt'],  # If NaN, keep original timestamp
                               df_complete['createdAt'] + pd.to_timedelta(df_complete['timezoneOffset'], unit='m'))  # Else, apply offset


df_complete["startTimestamp_day"] = df_complete.startTimestamp.dt.normalize()
df_complete["createdAt_day"] = df_complete.startTimestamp.dt.normalize()

df_complete["startTimestamp_hour"] = df_complete.startTimestamp.dt.hour
df_complete["createdAt_hour"] = df_complete.startTimestamp.dt.hour


In [16]:
df_loc_complete = df_complete[df_complete.type.isin(["Latitude", "Longitude"])]
df_loc_complete = df_loc_complete[["customer", "startTimestamp", "type", "doubleValue", 
                           'createdAt', 'startTimestamp_day', 'createdAt_day',
       'startTimestamp_hour', 'createdAt_hour']]

In [17]:
# Find the earliest 'startTimestamp_day' for each customer
earliest_timestamp_per_customer = df_loc_complete.groupby('customer')['startTimestamp_day'].min()

# Map the earliest timestamp back to the original dataframe
df_loc_complete['earliest_start_day'] = df_loc_complete['customer'].map(earliest_timestamp_per_customer)

# Calculate 'relative_day' as the difference in days from the earliest day
df_loc_complete['relative_day'] = (df_loc_complete['startTimestamp_day'] - df_loc_complete['earliest_start_day']).dt.days


In [18]:
today = pd.to_datetime('today').normalize()
df_loc_complete['potential_days_coverage'] = (today - df_loc_complete['earliest_start_day']).dt.days

# Count unique days with data for each customer
actual_days = df_loc_complete.groupby('customer')['startTimestamp_day'].nunique()

# Mapping the actual number of days back to the DataFrame
df_loc_complete['actual_days_with_data'] = df_loc_complete['customer'].map(actual_days)

df_loc_complete['data_coverage_per'] = (df_loc_complete['actual_days_with_data'] / df_loc_complete['potential_days_coverage']) * 100


In [19]:
df_pd_complete = df_complete[~df_complete.type.isin(["Latitude", "Longitude"])]
df_pd_complete = df_pd_complete[["customer", "startTimestamp", "type", "doubleValue", 
                           "timezoneOffset", 'createdAt', 'startTimestamp_day', 'createdAt_day',
       'startTimestamp_hour', 'createdAt_hour']]

In [20]:
# Find the earliest 'startTimestamp_day' for each customer
earliest_timestamp_per_customer = df_pd_complete.groupby('customer')['startTimestamp_day'].min()

# Map the earliest timestamp back to the original dataframe
df_pd_complete['earliest_start_day'] = df_pd_complete['customer'].map(earliest_timestamp_per_customer)

# Calculate 'relative_day' as the difference in days from the earliest day
df_pd_complete['relative_day'] = (df_pd_complete['startTimestamp_day'] - df_pd_complete['earliest_start_day']).dt.days


In [21]:
df_pd_complete['potential_days_coverage'] = (today - df_pd_complete['earliest_start_day']).dt.days

# Count unique days with data for each customer
actual_days = df_pd_complete.groupby('customer')['startTimestamp_day'].nunique()

# Mapping the actual number of days back to the DataFrame
df_pd_complete['actual_days_with_data'] = df_pd_complete['customer'].map(actual_days)

df_pd_complete['data_coverage_per'] = (df_pd_complete['actual_days_with_data'] / df_pd_complete['potential_days_coverage']) * 100


In [22]:
today = "22042024"

with open(datapath + f'/passive_data_{today}.pkl', 'wb') as file:
    pickle.dump(df_pd_complete, file)
    
with open(datapath + f'/gps_data_{today}.pkl', 'wb') as file:
    pickle.dump(df_loc_complete, file)

### EMA data

In [23]:
# load data
session = pd.read_csv(datapath1 + "questionnaireSession.csv")

# session data
session["user"] = session["user"].str[:4]
session.rename(columns = {"user":"customer","completedAt": "quest_complete", "createdAt": "quest_create", "expirationTimestamp": "quest_expir"}, inplace=True)
session["quest_create"] = (pd.to_datetime(session["quest_create"],unit='ms'))
session["quest_complete"] = (pd.to_datetime(session["quest_complete"],unit='ms'))
df_sess = session[["customer", "sessionRun", "quest_create", "quest_complete", "study"]]

In [24]:
df_sess = pd.merge(df_sess, df_monitoring, on = "customer")

In [25]:
df_sess = df_sess.copy()
df_sess["quest_create_day"] = df_sess.quest_create.dt.normalize()
df_sess["quest_complete_day"] = df_sess.quest_complete.dt.normalize()

df_sess["quest_create_hour"] = df_sess.quest_create.dt.hour
df_sess["quest_complete_hour"] = df_sess.quest_complete.dt.hour

In [26]:
# count number of completed EMA beeps in first phase
df_sess1 = df_sess.loc[df_sess.study.isin([24,25])]
df_sess1 = df_sess1.copy()

# Convert the 'ema_base_start' and 'quest_complete_day' columns to datetime
try:
    df_sess1['ema_start_date'] = pd.to_datetime(df_sess1['ema_start_date'])
    df_sess1['quest_complete_day'] = pd.to_datetime(df_sess1['quest_complete_day'])
except Exception as e:
    print(f"Error converting to datetime: {e}")

# Check if conversion was successful
if df_sess1['ema_start_date'].dtype == 'datetime64[ns]' and df_sess1['quest_complete_day'].dtype == 'datetime64[ns]':
    # Calculate 'day_relative'
    df_sess1['quest_complete_relative1'] = (df_sess1['quest_complete_day'] - df_sess1['ema_start_date']).dt.days
else:
    print("Failed to convert one or more date columns to datetime format.")

sess_count1 = df_sess1.dropna(subset=["quest_complete"]).groupby("customer")["quest_complete"].size()\
.reset_index()
sess_count1 = sess_count1.rename(columns = {"quest_complete":"nquest_EMA1"})


In [27]:
daily_counts = df_sess1.groupby(['customer', 'quest_complete_day','quest_complete_relative1']).size().reset_index(name='daily_entries_sum')

In [28]:
# count number of completed EMA beeps in second phase
df_sess2 = df_sess.loc[df_sess.study.isin([33,34])]
sess_count2 = df_sess2.dropna(subset=["quest_complete"]).groupby("customer")["quest_complete"].size()\
.reset_index()
sess_count2 = sess_count2.rename(columns = {"quest_complete":"nquest_EMA2"})

In [29]:
df_sess = df_sess.merge(sess_count1, on=['customer'], how='left')
df_sess = df_sess.merge(sess_count2, on=['customer'], how='left')

In [30]:
df_sess = df_sess.merge(daily_counts, on=['customer', "quest_complete_day"], how='left')

In [31]:

filtered_df_sess = df_sess[(df_sess['quest_complete_relative1'] >= 16) | (df_sess['quest_complete_relative1'] < 0)]

# Find the index of the maximum 'quest_complete_relative1' for each customer
idx = filtered_df_sess.groupby('customer')['quest_complete_relative1'].idxmax()

# Select the rows that correspond to the maximum 'quest_complete_relative1' for each customer
max_quest_complete_relative1 = filtered_df_sess.loc[idx]

# Display the rows with relevant columns
max_quest_complete_relative1[['customer', 'for_id', 'quest_complete_relative1', 'quest_complete_day', 'quest_create_day','ema_start_date']]


,customer,for_id,quest_complete_relative1,quest_complete_day,quest_create_day,ema_start_date
502,5qL5,FOR14903,20.0,2023-07-02,2023-07-02,2023-06-12
2165,BdSf,FOR12902,20.0,2023-07-31,2023-07-31,2023-07-11
3291,GjiG,FOR12005,25.0,2023-08-28,2023-08-28,2023-08-03
4211,M72F,FOR12007,20.0,2023-09-18,2023-09-18,2023-08-29
388,N3CY,FOR11003,-1.0,2023-06-05,2023-06-05,2023-05-31
6412,bJlI,FOR13012,36.0,2023-10-25,2023-10-25,2023-09-19
3069,nrqZ,FOR11026,20.0,2023-08-22,2023-08-22,2023-08-02
5118,p4A1,FOR12010,20.0,2023-10-05,2023-10-05,2023-09-15


In [32]:
today = "22042024"

with open(datapath + f'/ema_data_{today}.pkl', 'wb') as file:
    pickle.dump(df_sess, file)
    
with open(datapath + f'/monitoring_data_{today}.pkl', 'wb') as file:
    pickle.dump(df_monitoring, file)